In [88]:
import re
from typing import List

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import explode, udf, size, col, concat_ws, to_date, to_timestamp, date_trunc, count, array_contains, month, dayofmonth, hour, explode_outer
from pyspark.sql.types import StringType, ArrayType, Row

In [40]:
spark = SparkSession.builder.appName('practice').config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.30').master(
    "local").getOrCreate()

In [65]:
def write_db(save_df: DataFrame, table_name: str, mode="overwrite"):
    save_df.write.format("jdbc") \
        .mode(mode) \
        .option("url", "jdbc:mysql://127.0.0.1:3306/cs179g") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .option("dbtable", f"{table_name}") \
        .option("user", "group6") \
        .option("batchsize", "100000") \
        .option("password", "grp6").save()

In [77]:
stocks_monthly_df: DataFrame = spark.read \
    .option("multiLine", "true") \
    .option("mode", "PERMISSIVE") \
    .json("stock_jsons/stocks_monthly*")

In [42]:
stocks_monthly_df.printSchema()

root
 |-- close: double (nullable = true)
 |-- date: string (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- ticker: string (nullable = true)
 |-- transactions: long (nullable = true)
 |-- volume: double (nullable = true)
 |-- vwap: double (nullable = true)



In [56]:
stocks_monthly_df=stocks_monthly_df.withColumn('date',to_date('date'))
stocks_monthly_df=stocks_monthly_df.withColumn('month',month('date')).withColumn('day',dayofmonth('date')).drop('date')
stocks_monthly_df.count()

In [66]:
#write_db(stocks_monthly_df,'stocks_monthly')

In [89]:
stocks_hourly_df: DataFrame = spark.read \
    .option("multiLine", "true") \
    .option("mode", "PERMISSIVE") \
    .json("stock_jsons/stocks_hourly*")

In [80]:
stocks_hourly_df.printSchema()

root
 |-- close: double (nullable = true)
 |-- date: string (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- ticker: string (nullable = true)
 |-- time: string (nullable = true)
 |-- volume: double (nullable = true)



In [90]:
stocks_hourly_df=stocks_hourly_df.withColumn('date',to_date('date'))
stocks_hourly_df=stocks_hourly_df.withColumn('month',month('date')).withColumn('day',dayofmonth('date')).drop('date')
stocks_hourly_df=stocks_hourly_df.withColumn('time',hour('time')).withColumnRenamed('time','hour')

In [94]:
#write_db(stocks_hourly_df,'stocks_hourly')